In [10]:
# -*- coding: utf-8 -*-
"""

By Michael Fornito, Kyrsh Rejendran, Thomas Fiello

"""
#!pip3 install flightradar24
#!pip3 install --upgrade google-cloud
#!pip3 install --upgrade google-cloud-storage
#!pip3 install --upgrade google-cloud-bigquery
#!pip install --upgrade pandas_gbq
#!pip install pyarrow
#!pip install mysql-connector-python

from google.cloud import bigquery
import mysql.connector
import pandas as pd
import json
import flightradar24
import requests
import math
import os
from datetime import datetime
import geopandas as gpd
import geoplot as gplt
import folium
from branca.element import Figure
fr = fr = flightradar24.Api()

## Python Code for Populating OpenSky Data onto Google Cloud

### Find Airport Information

In [11]:
def findAirportLoc(name):
    #FlightRadar24
    dataAirport = fr.get_airports()
    listAirport= dataAirport['rows']
    for airport in listAirport:
        if(name in airport['name']):
            return airport['lat'], airport['lon']

### Set the distance around airport for OpenSky data

In [12]:
def findLocRange(lat_center,long_center):
    #20km range
    range = 0.09
    lat_min = lat_center - range
    lat_max = lat_center + range
    long_min = long_center - (range/ math.cos(lat_center*math.pi/180))
    long_max = long_center + (range / math.cos(lat_center*math.pi/180))
    return lat_min,lat_max,long_min,long_max

### Queries Opensky network with given lat and lon information and returns data as pandas dataframe

In [13]:
def collectData(lat_min, lat_max, long_min, long_max):
    r = requests.get('https://opensky-network.org/api/states/all?lamin='+lat_min+'&lomin='+long_min+'&lamax='+lat_max+'&lomax='+long_max)
    r.status_code
    file=r.json()

    df=pd.DataFrame(file['states'],columns=['icao24', 'callsign','origin_country','time_position','last_contact',
                                            'longitude','latitude','baro_altitude','on_ground','velocity',
                                            'true_track','vertical_rate','sensors','geo_altitude','squawk','spi','x','y']).drop(columns=['x','y']).assign(obs_time= lambda x: file['time'])
    df[['time_position','last_contact','obs_time']] = df[['time_position','last_contact','obs_time']].apply(pd.to_datetime, unit='s')
    #df.set_index(['obs_time', 'callsign'],inplace=True)
    return df

### Cleans Pandas DataFrame

In [14]:
def cleanDataframe(df):
    df = df.drop(columns=['sensors','squawk'])
    df = df.where(pd.notnull(df), None)
    df = df.reset_index(drop=True)
    return df

### Pushes cleaned DataFrame to the cloud

In [15]:
def connectQuery(df):
    credentials = r'C:\Users\tfiel\Documents\GitHub\ERAU_CS540_Team4\cs540-project-key.json'
    os.environ['Google_APPLICATION_CREDENTIALS'] = credentials
    client = bigquery.Client()
    table_id = 'cs540-project.Flights.AllData'
    df = df.to_gbq(table_id , if_exists='append')

### Test Purposes Only

In [16]:
def createTest(): #Test the database on the test table
    credentials = r'C:\Users\tfiel\Documents\GitHub\ERAU_CS540_Team4\cs540-project-key.json'
    os.environ['Google_APPLICATION_CREDENTIALS'] = credentials
    client = bigquery.Client()
    table_id = 'cs540-project.Flights.Test'
    rows_to_insert = [
        {u'Name' : 'Mike', u'Age': 23},
        {u'Name' : 'DummyData', u'Age': 24},
        ]
    errors = client.insert_rows_json(table_id,rows_to_insert)

### Main function which starts process

In [17]:
def main():
    
    lat_center, long_center = findAirportLoc('Daytona')
    lat_min, lat_max, long_min, long_max = findLocRange(lat_center, long_center)
    df = collectData(str(lat_min), str(lat_max), str(long_min), str(long_max))
    df = cleanDataframe(df)
    #connectQuery(df)
    return(df)
    
adsb_df = main()

### Build Map

In [19]:
fig=Figure(width=550,height=350)
m=folium.Map(width=550,height=350,location=[29.18, -81.05],zoom_start=11,min_zoom=8,max_zoom=14)
fig.add_child(m)
icon_url=r'C:\Users\tfiel\Documents\GitHub\plane_icon.png'
for i in range(0,len(adsb_df)):
    lat = adsb_df.loc[i]['latitude']
    lon = adsb_df.loc[i]['longitude']
    callsign = adsb_df.loc[i]['callsign']
    new_icon = folium.features.CustomIcon(icon_url, icon_size=(16,16), icon_anchor=('center'))
    folium.Marker([lat, lon], popup=callsign, icon=new_icon).add_to(m)
m